# Water Surface Extraction – Laguna de Fúquene (1985–2024)

This notebook extracts and quantifies the annual surface area of the Laguna de Fúquene using satellite imagery. The output of this notebook is a tabular dataset that will be used in subsequent analysis notebooks.

The focus here is **data extraction and methodological transparency**, not interpretation or trend analysis.

## 1. Objective

The objective of this notebook is to:
- Extract annual water surface area estimates for the Laguna de Fúquene
- Ensure temporal consistency across the full historical period (1985–2024)
- Generate a reproducible dataset suitable for downstream analysis

No conclusions or trend interpretations are produced at this stage.

## 2. Libraries and Earth Engine initialization

In [55]:
import ee
import pandas as pd
import numpy as np
import geemap
import json
import os


ee.Authenticate()
ee.Initialize(project="earth-observation-fuquene")

## 3. Area of study

The region of interest (ROI) corresponds to the Laguna de Fúquene, Colombia. A polygon covering the full historical extent of the lake is used to avoid systematic underestimation of surface area.

In [56]:
# Laguna de Fúquene ROI (editable)
roi_sistema = ee.Geometry.Polygon([
    [
        [-73.80, 5.50],
        [-73.80, 5.42],
        [-73.70, 5.42],
        [-73.70, 5.50],
    ]
])

ruta = r"C:\Users\luism\Documents\github\Data\roi_laguna_fuquene.geojson"

with open(ruta, "r", encoding="utf-8") as f:
    geojson = json.load(f)

coords = geojson["features"][0]["geometry"]["coordinates"]

roi_laguna = ee.Geometry.Polygon(coords)

#roi_laguna = geemap.geojson_to_ee(geojson)







In [57]:
Map = geemap.Map()
Map.centerObject(roi_laguna, 11)

# ROI sistema → rojo
Map.addLayer(
    roi_sistema,
    {'color': 'red'},
    'ROI Sistema hidrológico'
)

# ROI laguna → verde
Map.addLayer(
    roi_laguna,
    {'color': 'green'},
    'ROI Laguna oficial'
)

Map


Map(center=[5.463774214396926, -73.74551146157759], controls=(WidgetControl(options=['position', 'transparent_…

In [58]:



year = 2020

water_image = (
    ee.ImageCollection("JRC/GSW1_4/YearlyHistory")
    .filter(ee.Filter.eq('year', year))
    .first()
)

water_mask = water_image.eq(2)  # water present

Map = geemap.Map(center=[5.47, -73.77], zoom=11)

Map.addLayer(water_mask.updateMask(water_mask),
             {'palette': ['blue']},
             f'Agua detectada {year}')

Map.addLayer(roi_laguna, {'color': 'red'}, 'ROI')
#Map.addLayer(roi_sistema, {'color': 'brown'}, 'ROI')

Map

Map(center=[5.47, -73.77], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

## 4. Satellite data selection

This analysis relies on Landsat surface reflectance products to ensure long-term temporal consistency. Landsat 5, 7, and 8 collections are used depending on the year.

In [59]:
def get_landsat_collection(start, end):

    l5 = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
          .filterDate(start, end)
          .filterBounds(roi_sistema))

    l7 = (ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
          .filterDate(start, end)
          .filterBounds(roi_sistema))

    l8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
          .filterDate(start, end)
          .filterBounds(roi_sistema))

    return l5.merge(l7).merge(l8)



# Radiometric Scale

In [60]:
def apply_scale_factors(image):
    optical = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    return image.addBands(optical, None, True)


In [61]:
def get_landsat_collection(start, end):

    l5 = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
          .filterDate(start, end)
          .filterBounds(roi_sistema))

    l7 = (ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
          .filterDate(start, end)
          .filterBounds(roi_sistema))

    l8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
          .filterDate(start, end)
          .filterBounds(roi_sistema))

    return l5.merge(l7).merge(l8)


In [62]:
def mask_clouds(image):
    qa = image.select('QA_PIXEL')

    cloud = qa.bitwiseAnd(1 << 3).neq(0)
    shadow = qa.bitwiseAnd(1 << 4).neq(0)

    mask = cloud.Or(shadow).Not()

    return image.updateMask(mask)


In [63]:
def apply_scale_factors(image):
    optical = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    return image.addBands(optical, overwrite=True)

# NDWI (mezcla de sensores)

In [64]:
def add_ndwi(image):

    band_names = image.bandNames()

    green = ee.Image(
        ee.Algorithms.If(
            band_names.contains('SR_B3'),
            image.select('SR_B3'),   # Landsat 8
            image.select('SR_B2')    # Landsat 5/7
        )
    )

    nir = ee.Image(
        ee.Algorithms.If(
            band_names.contains('SR_B5'),
            image.select('SR_B5'),   # Landsat 8
            image.select('SR_B4')    # Landsat 5/7
        )
    )

    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')

    return image.addBands(ndwi)



## 5. Water detection method

Water surfaces are identified using a spectral water index (e.g., NDWI). A fixed threshold is applied consistently across all years to avoid introducing subjective adjustments.

In [65]:
years = list(range(1985, 2025))
water_area_data = []

for year in years:

    start = f'{year}-01-01'
    end = f'{year}-12-31'

    collection = (
        get_landsat_collection(start, end)
        .map(mask_clouds)
        .map(apply_scale_factors)
        .map(add_ndwi)
    )

    size = collection.size().getInfo()
    print(f'Año {year} | imágenes: {size}')

    if size == 0:
        continue

    composite = collection.select('NDWI').median()

    water_mask = composite.gt(0.1)   # umbral más robusto

    area_image = water_mask.multiply(ee.Image.pixelArea())

    # sistema hidrológico
    area_sistema = area_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi_sistema,
        scale=30,
        maxPixels=1e13
    )

    # laguna oficial
    area_laguna = area_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi_laguna,
        scale=30,
        maxPixels=1e13
    )

    sistema_ha = ee.Number(area_sistema.get('NDWI')).divide(10000)
    laguna_ha = ee.Number(area_laguna.get('NDWI')).divide(10000)

    water_area_data.append({
        'year': year,
        'area_sistema_ha': sistema_ha.getInfo(),
        'area_laguna_ha': laguna_ha.getInfo(),
        'Diferencia ' : sistema_ha.getInfo() - laguna_ha.getInfo()
    })



Año 1985 | imágenes: 4
Año 1986 | imágenes: 14
Año 1987 | imágenes: 9
Año 1988 | imágenes: 0
Año 1989 | imágenes: 2
Año 1990 | imágenes: 6
Año 1991 | imágenes: 18
Año 1992 | imágenes: 0
Año 1993 | imágenes: 9
Año 1994 | imágenes: 3
Año 1995 | imágenes: 3
Año 1996 | imágenes: 10
Año 1997 | imágenes: 8
Año 1998 | imágenes: 17
Año 1999 | imágenes: 27
Año 2000 | imágenes: 25
Año 2001 | imágenes: 29
Año 2002 | imágenes: 20
Año 2003 | imágenes: 15
Año 2004 | imágenes: 16
Año 2005 | imágenes: 18
Año 2006 | imágenes: 18
Año 2007 | imágenes: 20
Año 2008 | imágenes: 16
Año 2009 | imágenes: 21
Año 2010 | imágenes: 22
Año 2011 | imágenes: 21
Año 2012 | imágenes: 22
Año 2013 | imágenes: 34
Año 2014 | imágenes: 44
Año 2015 | imágenes: 42
Año 2016 | imágenes: 42
Año 2017 | imágenes: 42
Año 2018 | imágenes: 43
Año 2019 | imágenes: 42
Año 2020 | imágenes: 44
Año 2021 | imágenes: 42
Año 2022 | imágenes: 46
Año 2023 | imágenes: 42
Año 2024 | imágenes: 21


In [66]:
df = pd.DataFrame(water_area_data)
df.head()
df.tail()

,year,area_sistema_ha,area_laguna_ha,Diferencia
33,2020,547.459883,509.909120,37.550763
34,2021,188.066129,150.454634,37.611496
35,2022,51.889501,35.819474,16.070027
36,2023,239.599405,185.582880,54.016525
37,2024,113.924201,106.220360,7.703841


# Visualización en un año (específico)

In [67]:
year_example = 2015


collection_example = (
    get_landsat_collection(f'{year_example}-01-01', f'{year_example}-12-31')
    .map(apply_scale_factors)
    .map(add_ndwi)
)

composite_example = collection_example.select('NDWI').median()
water_mask_example = composite_example.gt(0)

Map = geemap.Map()
Map.centerObject(roi_laguna, 10)

Map.addLayer(
    composite_example,
    {'min': -1, 'max': 1, 'palette': ['brown', 'white', 'blue']},
    'NDWI'
)

Map.addLayer(
    water_mask_example,
    {'palette': ['grey']},
    'Water mask'
)

Map


Map(center=[5.463774214396926, -73.74551146157759], controls=(WidgetControl(options=['position', 'transparent_…

## 7. Output dataset

The final output of this notebook is a CSV file containing annual water surface area estimates. This dataset serves as the input for all subsequent analysis notebooks.

In [69]:
df = pd.DataFrame(water_area_data)

os.makedirs('Data', exist_ok=True)

df.to_csv(r'C:\Users\luism\Documents\github\Data\water_surface_area_fquene_1985_2024.csv',
    index=False
)

df.head()

,year,area_sistema_ha,area_laguna_ha,Diferencia
0,1985,2003.972804,1996.374178,7.598626
1,1986,1976.560175,1974.545934,2.014242
2,1987,1957.159307,1956.559660,0.599647
3,1989,1833.624063,1833.624063,0.000000
4,1990,1884.321868,1882.486419,1.835450
